In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import io
import logging
from datetime import timedelta,datetime,timezone

from faster_whisper import WhisperModel
from app.services.audio.redis import Transcript

from app.database_redis.connection import get_redis_client
from app.services.audio.audio import AudioSlicer
from app.services.audio.redis import Meeting, Transcriber, best_covering_connection
from app.settings import settings


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


check_and_process_connections_interval_sec: 1.0


In [3]:
overlap = 2

In [4]:
logger = logging.getLogger(__name__)

In [5]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [6]:
transcriber = Transcriber(redis_client)
meeting_id = await transcriber.pop_inprogress()

In [7]:
meeting_id = 'jrw-mdph-usx'

In [8]:
meeting = Meeting(redis_client, meeting_id)
await meeting.load_from_redis()

In [9]:
meeting.start_timestamp

datetime.datetime(2024, 5, 24, 13, 57, 21, 431494, tzinfo=tzutc())

In [10]:
meeting.transcriber_seek_timestamp

datetime.datetime(2024, 5, 24, 13, 57, 21, 431494, tzinfo=tzutc())

In [11]:
seek = meeting.transcriber_seek_timestamp - meeting.start_timestamp

In [12]:
seek

datetime.timedelta(0)

In [13]:
current_time = datetime.now(timezone.utc)

In [14]:
connections = await meeting.get_connections()

In [15]:
connection = best_covering_connection(meeting.diarizer_seek_timestamp, current_time, connections)

In [16]:
max_length = 240

In [17]:
connection = best_covering_connection(meeting.diarizer_seek_timestamp, current_time, connections)
audio_slicer = await AudioSlicer.from_ffmpeg_slice(f"/audio/{connection.id}.webm", seek, max_length)
slice_duration = audio_slicer.audio.duration_seconds
audio_data = await audio_slicer.export_data()

2024-05-24 14:19:41,207 - INFO - app.services.audio.audio - None


In [18]:
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [19]:
segments, _ = model.transcribe(io.BytesIO(audio_data), beam_size=5, vad_filter=True, word_timestamps=True)

In [20]:
segments = list(segments)

In [21]:
import pandas as pd

/tmp/ipykernel_3798/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [23]:
df = pd.DataFrame(segments)

In [24]:
df

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,words
0,1,2718,2.29,7.47,мы сейчас быстренько проверим и вернёмся к этому,"[50365, 4777, 10241, 37283, 3888, 3752, 30901,...",0.0,-0.424519,2.118056,0.264893,"[(2.29, 3.13, мы, 0.06396484375), (3.13, 4.79..."
1,2,2718,7.47,13.96,"и вернёмся к этому, быстренько проверим и вер...","[50615, 1006, 10544, 489, 12868, 1679, 981, 31...",0.0,-0.424519,2.118056,0.264893,"[(7.47, 8.67, и, 0.11700439453125), (9.98, 11..."
2,3,2718,17.19,18.45,"так, и чего?","[50865, 2936, 11, 1006, 19275, 30, 51115]",0.0,-0.424519,2.118056,0.264893,"[(17.19, 18.07, так,, 0.62353515625), (18.07,..."
3,4,2718,21.63,22.51,ну?,"[51115, 13087, 30, 51365]",0.0,-0.424519,2.118056,0.264893,"[(21.63, 22.51, ну?, 0.31201171875)]"
4,5,2718,30.11,47.13,"вот такое происходит, почему мы его не ловим?","[51365, 5505, 18292, 28548, 11, 21513, 4777, 6...",0.0,-0.424519,2.118056,0.264893,"[(30.11, 30.99, вот, 0.3994140625), (36.02, 3..."
5,6,5718,59.24,65.87,"странно, почему мы его не ловим?","[50365, 18057, 1234, 11, 21513, 4777, 6448, 17...",0.0,-0.399493,1.854015,0.005058,"[(59.24, 59.96, странно,, 0.173858642578125),..."
6,7,5718,66.05,66.81,это странно,"[50565, 2691, 18057, 1234, 50665]",0.0,-0.399493,1.854015,0.005058,"[(66.05, 66.11, это, 0.360595703125), (66.11,..."
7,8,5718,72.92,75.70,"так, модель у нас всё равно уже здесь, да?","[50665, 2936, 11, 24104, 3908, 1595, 6519, 964...",0.0,-0.399493,1.854015,0.005058,"[(72.92, 73.28, так,, 0.2705078125), (73.4, 7..."
8,9,5718,86.05,87.49,это нормальный,"[50815, 2691, 24068, 23393, 50965]",0.0,-0.399493,1.854015,0.005058,"[(86.05, 86.77, это, 0.347900390625), (86.77,..."
9,10,5718,87.49,96.47,я стану собакой,"[50965, 2552, 27214, 585, 10450, 1272, 1700, 5...",0.0,-0.399493,1.854015,0.005058,"[(87.49, 88.39, я, 0.045654296875), (94.95, 9..."


In [ ]:

segments = [s for s in list(segments)]
logger.info("done")
result = [[w._asdict() for w in s.words] for s in segments]
transcription = Transcript(meeting_id, redis_client, result)
await transcription.lpush()

In [57]:
''.join([w['word'] for w in result[-1]])

' Не чеши батон, открошит он, и раскрошит он твой, чьи-то дон, дон, дон, дон, дон, дон, дон, дон, дон, дон, дон, дон, дон, потому что пчёл не он, гусь к нему чёрный.'

In [45]:
segments

[Segment(id=1, seek=2332, start=28.4, end=111.29, text=' Ну, теперь-то я вам что-нибудь скажу, вам что-нибудь скажу, вам, вот теперь-то я тоже вам что-нибудь скажу, а вот прямо сейчас я вам прямо что-нибудь скажу.', tokens=[50365, 7571, 11, 16983, 12, 860, 2552, 10448, 2143, 12, 23561, 21938, 585, 11, 10448, 2143, 12, 23561, 21938, 585, 11, 10448, 11, 5505, 16983, 12, 860, 2552, 12251, 10448, 2143, 12, 23561, 21938, 585, 11, 2559, 5505, 28547, 10241, 2552, 10448, 28547, 2143, 12, 23561, 21938, 585, 13, 50975], temperature=0.0, avg_logprob=-0.39853897032799657, compression_ratio=2.3035714285714284, no_speech_prob=0.11944580078125, words=[Word(start=28.4, end=28.86, word=' Ну,', probability=0.470947265625), Word(start=28.88, end=29.1, word=' теперь', probability=0.93310546875), Word(start=29.1, end=29.28, word='-то', probability=0.810791015625), Word(start=29.28, end=29.48, word=' я', probability=0.955078125), Word(start=29.48, end=29.68, word=' вам', probability=0.79931640625), Word(sta

In [46]:
seek

datetime.timedelta(seconds=238, microseconds=48000)

In [47]:
meeting.transcriber_seek_timestamp = meeting.start_timestamp+seek +timedelta(seconds = slice_duration-overlap)
await transcriber.remove(meeting.meeting_id)
await meeting.update_redis()

In [48]:
meeting.transcriber_seek_timestamp

datetime.datetime(2024, 5, 22, 16, 2, 7, 966038, tzinfo=tzutc())

In [49]:
await transcriber.remove(meeting.meeting_id)

In [50]:
await meeting.update_redis()

In [51]:
slice_duration

240.048

In [52]:
segments

[Segment(id=1, seek=2332, start=28.4, end=111.29, text=' Ну, теперь-то я вам что-нибудь скажу, вам что-нибудь скажу, вам, вот теперь-то я тоже вам что-нибудь скажу, а вот прямо сейчас я вам прямо что-нибудь скажу.', tokens=[50365, 7571, 11, 16983, 12, 860, 2552, 10448, 2143, 12, 23561, 21938, 585, 11, 10448, 2143, 12, 23561, 21938, 585, 11, 10448, 11, 5505, 16983, 12, 860, 2552, 12251, 10448, 2143, 12, 23561, 21938, 585, 11, 2559, 5505, 28547, 10241, 2552, 10448, 28547, 2143, 12, 23561, 21938, 585, 13, 50975], temperature=0.0, avg_logprob=-0.39853897032799657, compression_ratio=2.3035714285714284, no_speech_prob=0.11944580078125, words=[Word(start=28.4, end=28.86, word=' Ну,', probability=0.470947265625), Word(start=28.88, end=29.1, word=' теперь', probability=0.93310546875), Word(start=29.1, end=29.28, word='-то', probability=0.810791015625), Word(start=29.28, end=29.48, word=' я', probability=0.955078125), Word(start=29.48, end=29.68, word=' вам', probability=0.79931640625), Word(sta

In [53]:

meeting.transcribe_seek_timestamp = end_of_last_speech + meeting.transcribe_seek_timestamp
await transcriber.remove(meeting.id)
await meeting.update_redis()

NameError: name 'end_of_last_speech' is not defined

In [54]:
logger = logging.getLogger(__name__)

# TODO: implement recurrent prompt from last item


async def process(redis_client, model, max_length=240) -> None:
    transcriber = Transcriber(redis_client)
    meeting_id = await transcriber.pop_inprogress()

    if not meeting_id:
        return

    meeting = Meeting(redis_client, meeting_id)
    await meeting.load_from_redis()
    seek = meeting.transcribe_seek_timestamp - meeting.start_timestamp

    connection = best_covering_connection(seek, meeting.start_timestamp, meeting.get_connections())
    audio_slicer = await AudioSlicer.from_ffmpeg_slice(f"/audio/{connection.id}.webm", seek, seek + max_length)
    audio_data = await audio_slicer.export_data()

    segments, _ = model.transcribe(io.BytesIO(audio_data), beam_size=5, vad_filter=True, word_timestamps=True)
    segments = [s for s in list(segments)]
    logger.info("done")
    result = [[w._asdict() for w in s.words] for s in segments]
    transcription = Transcript(meeting_id, redis_client, result)
    await transcription.lpush()
    end_of_last_speech = timedelta(seconds=result[-1][-1]["end"])

    meeting.transcribe_seek_timestamp = end_of_last_speech + meeting.transcribe_seek_timestamp
    await transcriber.remove(meeting.id)
    await meeting.update_redis()

In [ ]:


async def main():
    redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

    while True:
        await asyncio.sleep(0.1)
        await process(redis_client, model)


if __name__ == "__main__":
    model_size = "large-v3"
    model = WhisperModel(model_size, device="cuda", compute_type="float16")
    logger.info("Model loaded")
    asyncio.run(main())
